In [1]:
import os
import os.path as op
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import seaborn as sns
sns.set_theme(style="darkgrid", color_codes=True)
sns.set(font_scale=1.35, style="ticks") #set styling preferences
import statsmodels.api as sm
from scipy import stats
import math
from math import pi
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from scipy.spatial.distance import cdist
from scipy.cluster.vq import kmeans2,vq, whiten
import geopandas as gpd
import h5py
import boto.s3
import glob    
import boto3
from zipfile import ZipFile
import shutil
from pyspark.sql import SparkSession
import statsmodels.api as sm

In [2]:
# Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
# Showing the entire number in dataframe
pd.set_option('float_format', '{:f}'.format)

In [4]:
cols_to_use = ['IDMerged', 'tripIndex', 'actStartTime', 'actEndTime','duration_travelling', 'cost_BEAM', 'duration_walking', 'duration_in_privateCar', 'duration_on_bike', 'duration_in_ridehail', 
              'distance_travelling', 'duration_in_transit', 'distance_walking','distance_bike','distance_ridehail', 
              'distance_privateCar', 'distance_transit', 'mode_choice_planned_BEAM','mode_choice_actual_BEAM', 'vehicleIds',
              'numPassengers', 'replanning_status', 'reason', 'parkingType', 'actPurpose',
               'BlockGroupStart','startX', 'startY', 'bgid_start', 'tractid_start', 'juris_name_start', 'county_name_start', 'mpo_start', 
               'BlockGroupEnd', 'endX', 'endY', 'bgid_end', 'tractid_end', 'juris_name_end', 'county_name_end', 'mpo_end',  
               'duration_door_to_door', 'waitTime_no_replanning', 'waitTime_replanning',  
               'mode_choice_actual_6', 'mode_choice_actual_5', 'mode_choice_actual_4', 'telework', 'telework_option',
               'fuel_marginal', 'fuelGasoline', 'emissionGasoline', 'emission_marginal', 'job_sector', 
               'Potential_INEXUS_in_dollar_2023', 'Realized_INEXUS_in_dollar_2023', 'Realized_INEXUS', 
               'socialCarbonCost', 'mandatoryCat', 'income_quartiles', 'income_deciles','lever_position_tel',
               'income_in_thousands', 'hhsize', 'hh_workers', 'free_parking_at_work', 'distance_to_work',
               'roundtrip_auto_time_to_work', 'workplace_in_cbd', 'workplace_taz', 
               'trip_id','person_id','household_id','tour_id','primary_purpose_x','destination_logsum_x', 'trip_mode_AS_trips', 
               'logsum_trip_Potential_INEXUS', 'tour_mode_AS_tours','duration','earning','sex', 'race',
               'TAZ_x','home_x','home_y','home_taz','value_of_time', 'school_taz','workplace_taz', 'free_parking_at_work',
               'hh_size','hh_income','income_in_thousands','median_value_of_time', 'hh_value_of_time', 'home_is_urban',
               'home_is_rural', 'auto_ownership','age', 'income', 'hh_cars', 'origin_x', 'is_worker',
               'destination_x', 'TAZ_y', 'scenario', 'year','door_to_door_time_relative_to_baseline', 'tour_category',
               'potential_in_dollar_2023_relative_to_baseline', 'realized_in_dollar_2023_relative_to_baseline'] 
# Specify the columns to read

In [9]:
%%time
sf_stacked = pd.read_csv('gs://beam-core-analysis/sf_2018_telecommuting_cleaned.csv.gz', compression = 'gzip', usecols = cols_to_use)
#sf_2018_stacked_telecommuting_20230625

<timed exec>:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: total: 16min 34s
Wall time: 17min 31s


In [5]:
#sf_stacked[sf_stacked['reason']=='RouteNotAvailableForChosenMode WALK_TRANSIT'].groupby('lever_position_tel')['tripIndex'].count()

In [ ]:
#sf_stacked[sf_stacked['income'].isna()].groupby('lever_position_tel')['tripIndex'].count()

In [ ]:
#test4.groupby('lever_position_tel')['income'].isna().shape
#sf_stacked[sf_stacked['income'].isna()].groupby('lever_position_tel').size()

#### Calculate the INEXUS

In [ ]:
# Mapping BEAM and ASIM modes
conditions  = [(sf_stacked['mode_choice_actual_BEAM'] == 'walk')&(sf_stacked['trip_mode_AS_trips'] == 'WALK'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'bike')&(sf_stacked['trip_mode_AS_trips'] == 'BIKE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov3_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov3_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3PAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov3')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov3')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED3PAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov2_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov2_teleportation')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2PAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov2')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2PAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov2')&(sf_stacked['trip_mode_AS_trips'] == 'SHARED2FREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEFREE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'car')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEPAY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'walk_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_LRF'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'walk_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_LOC'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'walk_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_EXP'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'walk_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_HVY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'walk_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK_COM'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail')&(sf_stacked['trip_mode_AS_trips'] == 'TNC_SINGLE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail')&(sf_stacked['trip_mode_AS_trips'] == 'TNC_SHARED'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail')&(sf_stacked['trip_mode_AS_trips'] == 'TAXI'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'drive_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_COM'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'drive_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LRF'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'drive_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LOC'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'drive_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_EXP'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'drive_transit')&(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_HVY'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail_pooled')&(sf_stacked['trip_mode_AS_trips'] == 'TNC_SINGLE'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail_pooled')&(sf_stacked['trip_mode_AS_trips'] == 'TNC_SHARED'),
               (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail_pooled')&(sf_stacked['trip_mode_AS_trips'] == 'TAXI'),
       
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov2_teleportation')&((sf_stacked['trip_mode_AS_trips'] == 'SHARED3FREE')|
               (sf_stacked['trip_mode_AS_trips'] == 'SHARED3PAY')|(sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEFREE')|
               (sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEPAY')),
               
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov2')&((sf_stacked['trip_mode_AS_trips'] == 'SHARED3PAY')|
               (sf_stacked['trip_mode_AS_trips'] == 'SHARED3FREE')|(sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEFREE')|
               (sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEPAY')),
               
               (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov3')&((sf_stacked['trip_mode_AS_trips'] == 'SHARED2PAY')|
               (sf_stacked['trip_mode_AS_trips'] == 'SHARED2FREE')|(sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEFREE')|
               (sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEPAY')),   
               
               (sf_stacked['mode_choice_actual_BEAM'] == 'hov3_teleportation')&((sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEPAY')|
               (sf_stacked['trip_mode_AS_trips'] == 'DRIVEALONEFREE')|(sf_stacked['trip_mode_AS_trips'] == 'SHARED2PAY')|(sf_stacked['trip_mode_AS_trips'] == 'SHARED2FREE')),
               
               (sf_stacked['mode_choice_actual_BEAM'] == 'car')&((sf_stacked['trip_mode_AS_trips'] == 'SHARED3FREE')|(sf_stacked['trip_mode_AS_trips'] == 'SHARED2FREE')|
               (sf_stacked['trip_mode_AS_trips'] == 'SHARED3PAY')|(sf_stacked['trip_mode_AS_trips'] == 'SHARED2PAY')|(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LOC')|
               (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_LRF')|(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_EXP')|
               (sf_stacked['trip_mode_AS_trips'] == 'DRIVE_HVY')|(sf_stacked['trip_mode_AS_trips'] == 'DRIVE_COM')),
               
               (sf_stacked['mode_choice_actual_BEAM'] == 'walk')&((sf_stacked['trip_mode_AS_trips'] == 'WALK_LRF')|
               (sf_stacked['trip_mode_AS_trips'] == 'WALK_LOC')|(sf_stacked['trip_mode_AS_trips'] == 'WALK_EXP')|
               (sf_stacked['trip_mode_AS_trips'] == 'WALK_HVY')|(sf_stacked['trip_mode_AS_trips'] == 'WALK_COM')),
               
               (sf_stacked['mode_choice_actual_BEAM'] == 'walk_transit')&(sf_stacked['trip_mode_AS_trips'] == 'WALK'),
               
               ((sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail_pooled')|(sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail'))
               &(~((sf_stacked['trip_mode_AS_trips'] == 'TAXI')|(sf_stacked['trip_mode_AS_trips'] == 'TNC_SHARED')|
                (sf_stacked['trip_mode_AS_trips'] == 'TNC_SINGLE'))),
               
               (sf_stacked['mode_choice_actual_BEAM'] == 'walk_transit')&(~((sf_stacked['trip_mode_AS_trips'] == 'WALK_LRF')|
               (sf_stacked['trip_mode_AS_trips'] == 'WALK_LOC')|(sf_stacked['trip_mode_AS_trips'] == 'WALK_EXP')|
               (sf_stacked['trip_mode_AS_trips'] == 'WALK_HVY')|(sf_stacked['trip_mode_AS_trips'] == 'WALK_COM')|
               (sf_stacked['trip_mode_AS_trips'] == 'WALK'))),
               
               (sf_stacked['mode_choice_actual_BEAM'] == 'bike_transit'),
               
               (sf_stacked['mode_choice_actual_BEAM'] == 'car')&((sf_stacked['trip_mode_AS_trips'] == 'TNC_SHARED')|
               (sf_stacked['trip_mode_AS_trips'] == 'TNC_SINGLE')|(sf_stacked['trip_mode_AS_trips'] == 'TAXI'))]
 
                                                                      
                                                                                                                 
                                                                      
choices = [sf_stacked['WALK'], sf_stacked['BIKE'], sf_stacked['SHARED3FREE'], sf_stacked['SHARED3PAY'],
           sf_stacked['SHARED3FREE'], sf_stacked['SHARED3PAY'], sf_stacked['SHARED2FREE'], 
           sf_stacked['SHARED2PAY'], sf_stacked['SHARED2PAY'], sf_stacked['SHARED2FREE'], 
           sf_stacked['DRIVEALONEFREE'], sf_stacked['DRIVEALONEPAY'], sf_stacked['WALK_LRF'], 
           sf_stacked['WALK_LOC'], sf_stacked['WALK_EXP'], sf_stacked['WALK_HVY'], sf_stacked['WALK_COM'],
           sf_stacked['TNC_SINGLE'], 
           sf_stacked['TNC_SHARED'], sf_stacked['TAXI'],
           sf_stacked['DRIVE_COM'], sf_stacked['DRIVE_LRF'], sf_stacked['DRIVE_LOC'], sf_stacked['DRIVE_EXP'],
           sf_stacked['DRIVE_HVY'], sf_stacked['TNC_SINGLE'], sf_stacked['TNC_SHARED'], sf_stacked['TAXI'],
          sf_stacked[['DRIVEALONEFREE','DRIVEALONEPAY','SHARED2FREE','SHARED3FREE','SHARED2PAY','SHARED3PAY','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1),
          sf_stacked[['DRIVEALONEFREE','DRIVEALONEPAY','SHARED2FREE','SHARED3FREE','SHARED2PAY','SHARED3PAY','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1),
          sf_stacked[['DRIVEALONEFREE','DRIVEALONEPAY','SHARED2FREE','SHARED3FREE','SHARED2PAY','SHARED3PAY','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1),
          sf_stacked[['DRIVEALONEFREE','DRIVEALONEPAY','SHARED2FREE','SHARED3FREE','SHARED2PAY','SHARED3PAY','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1),
          sf_stacked[['DRIVEALONEFREE','DRIVEALONEPAY','SHARED2FREE','SHARED3FREE','SHARED2PAY','SHARED3PAY','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1),
          sf_stacked[['WALK','WALK_LOC','WALK_LRF','WALK_EXP','WALK_HVY','WALK_COM']].max(axis=1),
          sf_stacked[['WALK','WALK_LOC','WALK_LRF','WALK_EXP','WALK_HVY','WALK_COM']].max(axis=1),
          sf_stacked[['TAXI','TNC_SINGLE','TNC_SHARED']].max(axis=1),
          sf_stacked[['WALK','WALK_LOC','WALK_LRF','WALK_EXP','WALK_HVY','WALK_COM','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1),
          sf_stacked[['BIKE','WALK_LOC','WALK_LRF','WALK_EXP','WALK_HVY','WALK_COM','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1),
          sf_stacked[['DRIVEALONEFREE','DRIVEALONEPAY','SHARED2FREE','SHARED3FREE','SHARED2PAY','SHARED3PAY','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1)]

In [ ]:
sf_stacked['Realized_INEXUS'] = np.select(conditions, choices, default=np.nan)

In [ ]:
sf_stacked['Realized_INEXUS'] = np.where((sf_stacked['Realized_INEXUS'].isna())&((sf_stacked['mode_choice_actual_BEAM'] == 'car_hov2')|
                                          (sf_stacked['mode_choice_actual_BEAM'] == 'car_hov3')|
                                          (sf_stacked['mode_choice_actual_BEAM'] == 'hov2_teleportation')|
                                          (sf_stacked['mode_choice_actual_BEAM'] == 'hov3_teleportation')|
                                          (sf_stacked['mode_choice_actual_BEAM'] == 'car')|
                                          (sf_stacked['mode_choice_actual_BEAM'] == 'drive_transit')), 
                                           sf_stacked[['DRIVEALONEFREE','DRIVEALONEPAY','SHARED2FREE','SHARED3FREE','SHARED2PAY','SHARED3PAY','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1), sf_stacked['Realized_INEXUS'])

In [ ]:
sf_stacked['Realized_INEXUS'] = np.where((sf_stacked['Realized_INEXUS'].isna())&((sf_stacked['mode_choice_actual_BEAM'] == 'walk')|
                                         (sf_stacked['mode_choice_actual_BEAM'] == 'walk_transit')), 
                                          sf_stacked[['WALK','WALK_LOC','WALK_LRF','WALK_EXP','WALK_HVY','WALK_COM','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM']].max(axis=1),sf_stacked['Realized_INEXUS'])

In [ ]:
sf_stacked['Realized_INEXUS'] = np.where(sf_stacked['Realized_INEXUS'].isna()&(sf_stacked['mode_choice_actual_BEAM'] == 'bike'), 
                                         sf_stacked[['BIKE','WALK','WALK_LOC','WALK_LRF','WALK_EXP','WALK_HVY','WALK_COM']].max(axis=1),sf_stacked['Realized_INEXUS'])

In [ ]:
#sf_stacked2 = sf_stacked[~((sf_stacked['Realized_INEXUS'].isna()) & ((sf_stacked['reason'] == 'RouteNotAvailableForChosenMode RIDE_HAIL')|
                                                                    #(sf_stacked['reason'] == 'RouteNotAvailableForChosenMode WALK_TRANSIT')|
                                                                    #(sf_stacked['reason'] == 'RouteNotAvailableForChosenMode RIDE_HAIL_POOLED')|
                                                                    #(sf_stacked['reason'] == 'RouteNotAvailableForChosenMode BIKE')|
                                                                    #(sf_stacked['reason'] == 'RouteNotAvailableForChosenMode CAR')|
                                                                    #(sf_stacked['reason'] == 'RouteNotAvailableForChosenMode DRIVE_TRANSIT')))]

In [ ]:
# dropping the rows with NAN utility values
sf_stacked.dropna(subset=['Realized_INEXUS'], inplace=True)

In [ ]:
conditions  = [(sf_stacked['primary_purpose_x'] == 'work'),(sf_stacked['primary_purpose_x'] == 'univ'),
               (sf_stacked['primary_purpose_x'] == 'school'),(sf_stacked['primary_purpose_x'] == 'escort'),
              (sf_stacked['primary_purpose_x'] == 'shopping'),(sf_stacked['primary_purpose_x'] == 'eatout'),
              (sf_stacked['primary_purpose_x'] == 'othmaint'),(sf_stacked['primary_purpose_x'] == 'social'),
              (sf_stacked['primary_purpose_x'] == 'othdiscr'),(sf_stacked['primary_purpose_x'] == 'atwork')]
 
                                                                   
choices = [-0.022, -0.0271, -0.0271, -0.0279, -0.0279, -0.0279, -0.0175, -0.0175, -0.0279, -0.0279]

In [ ]:
sf_stacked['c_ivt'] = np.select(conditions, choices, default=np.nan)

In [ ]:
%%time 
#this alpha will convert the INEXUS values to cents per minute
sf_stacked['alpha'] = -0.6 *(sf_stacked['c_ivt'])/(sf_stacked['value_of_time'])

In [ ]:
sf_stacked['Realized_INEXUS_in_dollar_2023'] = ((sf_stacked['Realized_INEXUS']/sf_stacked['alpha'])/100)*1.75

In [ ]:
sf_stacked['Potential_INEXUS_in_dollar_2023'] = ((sf_stacked['logsum_trip_Potential_INEXUS']/sf_stacked['alpha'])/100)*1.75

In [ ]:
%%time
sf_stacked['realized_in_dollar_2023_relative_to_baseline'] = sf_stacked['Realized_INEXUS_in_dollar_2023'] - (sf_stacked['Realized_INEXUS_in_dollar_2023'].where(sf_stacked['lever_position_tel'].eq(1.0))
                                                                                                             .groupby([sf_stacked['IDMerged'],sf_stacked['tripIndex']])
                                                                                                             .transform('first')
                                                                                                             .convert_dtypes()) 

In [ ]:
%%time
sf_stacked['potential_in_dollar_2023_relative_to_baseline'] = sf_stacked['Potential_INEXUS_in_dollar_2023'] - (sf_stacked['Potential_INEXUS_in_dollar_2023'].where(sf_stacked['lever_position_tel'].eq(1.0))
                                                                                                  .groupby([sf_stacked['IDMerged'],sf_stacked['tripIndex']])
                                                                                                   .transform('first')
                                                                                                   .convert_dtypes()) 

#### Clean the data

In [ ]:
# 800k of the rows are dropped
sf_stacked_cl = sf_stacked[(sf_stacked['duration_walking'] < 3600)&(sf_stacked['distance_walking'] < 5000)]

In [ ]:
# drop unused columns
sf_stacked_cl = sf_stacked_cl.drop(['relate','WALK_EXP','WALK_HVY','WALK_COM','DRIVE_LOC','DRIVE_LRF','DRIVE_EXP','DRIVE_HVY','DRIVE_COM','TAXI','TNC_SINGLE','TNC_SHARED','DRIVEALONEFREE','DRIVEALONEPAY','SHARED2FREE','SHARED2PAY','SHARED3FREE','SHARED3PAY','WALK','BIKE','WALK_LOC','WALK_LRF'], axis = 1)

In [ ]:
sf_stacked_cl.shape

In [ ]:
# Filter out rows with errors  
sf_stacked_cle = sf_stacked_cl[(sf_stacked_cl['actPurpose'] != 'school_to_')&
                             (sf_stacked_cl['actPurpose'] != 'Home_to_')&
                             (sf_stacked_cl['actPurpose'] != 'work_to_')&
                             (sf_stacked_cl['actPurpose'] != 'shopping_to_')&
                             (sf_stacked_cl['actPurpose'] != 'social_to_')&
                             (sf_stacked_cl['actPurpose'] != 'othdiscr_to_')&
                             (sf_stacked_cl['actPurpose'] != 'othmaint_to_')&
                             (sf_stacked_cl['actPurpose'] != 'eatout_to_')&
                             (sf_stacked_cl['actPurpose'] != 'univ_to_')&
                             (sf_stacked_cl['actPurpose'] != 'escort_to_')&
                             (sf_stacked_cl['actPurpose'] != 'atwork_to_')]

In [ ]:
%%time
import gzip
from io import BytesIO
from google.cloud import storage
import pandas as pd

# Export DataFrame to a gzip-compressed CSV file
compressed_bytes = BytesIO()
with gzip.GzipFile(fileobj=compressed_bytes, mode='w') as gz:
    sf_stacked_cl.to_csv(gz, index=False)
compressed_bytes.seek(0)

# Upload the gzip-compressed file to Google Cloud Storage
storage_client = storage.Client.from_service_account_json('C:/Users/nrezaei/Documents/beam-core-a9ea929e82b1.json')
bucket_name = 'beam-core-analysis'  # Replace with your Google Cloud Storage bucket name
destination_blob_name = 'sf_2018_telecommuting_cleaned.csv.gz'  # Replace with the desired destination file name in the bucket

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_file(compressed_bytes, content_type='application/gzip')

# Set the Content-Encoding metadata property
blob.metadata = {'Content-Encoding': 'gzip'}
blob.patch()

#### More Cleaning

In [11]:
# shift column 'lever_position_tel' to first position
first_column = sf_stacked.pop('lever_position_tel')
# insert column using insert(position,column_name, first_column) function
sf_stacked.insert(7, 'lever_position_tel', first_column)

In [13]:
# shift column 'telework' to first position
second_column = sf_stacked.pop('telework')
# insert column using insert(position,column_name, second_column) function
sf_stacked.insert(8, 'telework', second_column)

In [15]:
sf_stacked['actStartTime_24h'] = sf_stacked['actStartTime']/3600

In [16]:
sf_stacked['actEndTime_24h'] = sf_stacked['actEndTime']/3600

In [17]:
sf_stacked['duration_travelling_min'] = sf_stacked['duration_travelling'] / 60

In [18]:
sf_stacked['distance_travelling_km'] = sf_stacked['distance_travelling'] / 1000

In [19]:
sf_stacked['distance_travelling_mile'] = sf_stacked['distance_travelling'] / 1609

In [20]:
# shift column 'actEndTime_24h' to first position
third_column = sf_stacked.pop('actEndTime_24h')
# insert column using insert(position,column_name, second_column) function
sf_stacked.insert(2, 'actEndTime_24h', third_column)

In [21]:
# shift column 'actStartTime_24h' to first position
fourth_column = sf_stacked.pop('actStartTime_24h')
# insert column using insert(position,column_name, second_column) function
sf_stacked.insert(3, 'actStartTime_24h', fourth_column)

In [22]:
# shift column 'duration_travelling_min' to first position
fifth_column = sf_stacked.pop('duration_travelling_min')
# insert column using insert(position,column_name, second_column) function
sf_stacked.insert(4, 'duration_travelling_min', fifth_column)

In [34]:
# shift column 'duration_travelling' to first position
sixth_column = sf_stacked.pop('duration_travelling')
# insert column using insert(position,column_name, second_column) function
sf_stacked.insert(109, 'duration_travelling', sixth_column)

In [24]:
# shift column 'actEndTime' to first position
seventh_column = sf_stacked.pop('actEndTime')
# insert column using insert(position,column_name, second_column) function
sf_stacked.insert(112, 'actEndTime', seventh_column)

In [25]:
# shift column 'actEndTime' to first position
eighth_column = sf_stacked.pop('actStartTime')
# insert column using insert(position,column_name, second_column) function
sf_stacked.insert(113, 'actStartTime', eighth_column)

In [26]:
# shift column 'actPurpose' to first position
nine_column = sf_stacked.pop('actPurpose')
# insert column using insert(position,column_name, second_column) function
sf_stacked.insert(6, 'actPurpose', nine_column)

In [30]:
# shift column 'distance_travelling_mile' to first position
tenth_column = sf_stacked.pop('distance_travelling_mile')
# insert column using insert(position,column_name, second_column) function
sf_stacked.insert(11, 'distance_travelling_mile', tenth_column)

In [32]:
# shift column 'duration_walking' to first position
eleventh_column = sf_stacked.pop('duration_walking')
# insert column using insert(position,column_name, second_column) function
sf_stacked.insert(14, 'duration_walking', eleventh_column)

In [20]:
# If we want to check which person ids have at least one trip in the baseline
# Filter the dataframe to include only rows where 'lever_position_tel' is equal to 1
#baseline = sf_stacked[sf_stacked['lever_position_tel'] == 1]
# Select the unique person IDs from the filtered dataframe
#unique_person_ids = baseline['IDMerged'].unique()
# Filter the dataframe to include only rows with person IDs matching unique_person_ids  #303490 trips are dropped because they don't have any baseline
#sf_stacked_filtered = sf_stacked[sf_stacked['IDMerged'].isin(unique_person_ids)]

In [21]:
# Selecting the ones who don't have any trips in the baseline scenario
#sf_stacked_filtered2 = sf_stacked[~sf_stacked['IDMerged'].isin(unique_person_ids)]

In [68]:
# Filter the dataframe to include only rows with the specified actPurpose values
#filtered_rows = sf_stacked[sf_stacked['actPurpose'].isin(['work_to_Home', 'Home_to_work'])].reset_index(drop=True)

In [76]:
# Get the unique person IDs with the target actPurpose values
#unique_person_ids = filtered_rows['IDMerged'].unique()

In [78]:
# Filter the original dataframe using the person IDs with the target actPurpose values  #6482606 rows
#sf_stacked_filtered = sf_stacked[sf_stacked['IDMerged'].isin(unique_person_ids)]

In [87]:
#sf_stacked_any_work = sf_stacked_filtered.sort_values(['IDMerged', 'lever_position_tel','actEndTime_24h']).reset_index(drop=True)

#### Adding two new rows to each person who teleworks

In [38]:
# Create an empty list to store the appended rows
appended_rows = []

# Group by person_id and check if 'telework' is True
for _, group_df in sf_stacked.groupby(['IDMerged', 'lever_position_tel']):
    if group_df['telework'].any():  # Check if 'telework' is True
        new_row = {'IDMerged': group_df['IDMerged'].iloc[0], 'mode_choice_actual_BEAM': 'telecommuting', 'duration_travelling': 0,
                  'duration_travelling_min': 0, 'actPurpose': 'Home_to_work', 'cost_BEAM': 0, 'lever_position_tel': group_df['lever_position_tel'].iloc[0], 
                  'telework':True, 'distance_travelling_mile': 0, 'fuelGasoline':0, 'mode_choice_actual_4': 'telecommuting',
                  'Potential_INEXUS_in_dollar_2023': 0, 'Realized_INEXUS_in_dollar_2023': 0, 'actEndTime_24h':0, 'actStartTime_24h':0,
                  'emissionGasoline':0, 'emission_marginal': 0, 'duration_door_to_door':0, 'waitTime_no_replanning':0, 'telework_option': True,
                   'waitTime_replanning':0, 'mode_choice_actual_5':'telecommuting', 'mode_choice_actual_6':'telecommuting','socialCarbonCost':0,
                   'duration_in_privateCar':0, 'duration_in_ridehail':0, 'duration_in_transit':0, 'duration_walking':0,
                   'replanning_status': 0,'income_deciles': group_df['income_deciles'].iloc[0], 'income_quartiles': group_df['income_quartiles'].iloc[0]}
        new_row2 = {'IDMerged': group_df['IDMerged'].iloc[0], 'mode_choice_actual_BEAM': 'telecommuting', 'duration_travelling': 0,
                  'duration_travelling_min': 0, 'actPurpose': 'work_to_Home', 'cost_BEAM': 0, 'lever_position_tel': group_df['lever_position_tel'].iloc[0], 
                  'telework':True, 'distance_travelling_mile': 0, 'fuelGasoline':0, 'mode_choice_actual_4': 'telecommuting',
                  'Potential_INEXUS_in_dollar_2023': 0, 'Realized_INEXUS_in_dollar_2023': 0, 'actEndTime_24h':0, 'actStartTime_24h':0,
                    'emissionGasoline':0, 'emission_marginal': 0, 'duration_door_to_door':0, 'waitTime_no_replanning':0, 'socialCarbonCost':0, 
                   'waitTime_replanning':0, 'mode_choice_actual_5':'telecommuting', 'mode_choice_actual_6':'telecommuting', 'telework_option': True,
                  'duration_in_privateCar':0, 'duration_in_ridehail':0, 'duration_in_transit':0, 'duration_walking':0,
                  'replanning_status': 0, 'income_deciles': group_df['income_deciles'].iloc[0], 'income_quartiles': group_df['income_quartiles'].iloc[0]}
        appended_rows.extend([new_row, new_row2])

  
# Create a dataframe from the appended rows
appended_df = pd.DataFrame(appended_rows)

# Append the newly created rows to the original dataframe
sf_stacked_new_rows = pd.concat([sf_stacked, appended_df], ignore_index=True).reset_index(drop=True)

In [39]:
sf_stacked_with_new_rows = sf_stacked_new_rows.sort_values(['IDMerged', 'lever_position_tel','actEndTime_24h']).reset_index(drop=True)

In [43]:
# Filter the dataframe to include only rows with the specified actPurpose values #3330626 rows
sf_tel_home_work = sf_stacked_with_new_rows[sf_stacked_with_new_rows['actPurpose'].isin(['work_to_Home', 'Home_to_work'])].reset_index(drop=True)

In [55]:
sf_tel_home_work_small = sf_tel_home_work[['IDMerged','tripIndex','actEndTime_24h','actStartTime_24h','duration_travelling_min','actPurpose','cost_BEAM',
                                           'lever_position_tel','telework','distance_travelling_mile','duration_in_privateCar','duration_in_ridehail',
                                           'duration_walking','mode_choice_actual_BEAM', 'replanning_status','mode_choice_actual_4',
                                           'Potential_INEXUS_in_dollar_2023', 'Realized_INEXUS_in_dollar_2023','income_quartiles', 'income_deciles',
                                          'income_in_thousands', 'income']]

In [56]:
sf_tel_home_work_small.to_csv('C:/Users/nrezaei/Documents/Data/sf_tel_reg.csv', index = False)

In [51]:
%%time
import gzip
from io import BytesIO
from google.cloud import storage
import pandas as pd

# Export DataFrame to a gzip-compressed CSV file
compressed_bytes = BytesIO()
with gzip.GzipFile(fileobj=compressed_bytes, mode='w') as gz:
    sf_stacked_with_new_rows.to_csv(gz, index=False)
compressed_bytes.seek(0)

# Upload the gzip-compressed file to Google Cloud Storage
storage_client = storage.Client.from_service_account_json('C:/Users/nrezaei/Documents/beam-core-a9ea929e82b1.json')
bucket_name = 'beam-core-analysis'  # Replace with your Google Cloud Storage bucket name
destination_blob_name = 'sf_2018_telecommuting_cleaned_with_telecommuting_rows.csv.gz'  # Replace with the desired destination file name in the bucket

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_file(compressed_bytes, content_type='application/gzip')

# Set the Content-Encoding metadata property
blob.metadata = {'Content-Encoding': 'gzip'}
blob.patch()

CPU times: total: 34min 20s
Wall time: 34min 33s


In [52]:
%%time
import gzip
from io import BytesIO
from google.cloud import storage
import pandas as pd

# Export DataFrame to a gzip-compressed CSV file
compressed_bytes = BytesIO()
with gzip.GzipFile(fileobj=compressed_bytes, mode='w') as gz:
    sf_tel_home_work.to_csv(gz, index=False)
compressed_bytes.seek(0)

# Upload the gzip-compressed file to Google Cloud Storage
storage_client = storage.Client.from_service_account_json('C:/Users/nrezaei/Documents/beam-core-a9ea929e82b1.json')
bucket_name = 'beam-core-analysis'  # Replace with your Google Cloud Storage bucket name
destination_blob_name = 'sf_2018_telecommuting_home_work.csv.gz'  # Replace with the desired destination file name in the bucket

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_file(compressed_bytes, content_type='application/gzip')

# Set the Content-Encoding metadata property
blob.metadata = {'Content-Encoding': 'gzip'}
blob.patch()

CPU times: total: 4min 59s
Wall time: 5min


In [57]:
%%time
import gzip
from io import BytesIO
from google.cloud import storage
import pandas as pd

# Export DataFrame to a gzip-compressed CSV file
compressed_bytes = BytesIO()
with gzip.GzipFile(fileobj=compressed_bytes, mode='w') as gz:
    sf_stacked_with_new_rows.to_csv(gz, index=False)
compressed_bytes.seek(0)

# Upload the gzip-compressed file to Google Cloud Storage
storage_client = storage.Client.from_service_account_json('C:/Users/nrezaei/Documents/beam-core-a9ea929e82b1.json')
bucket_name = 'beam-core-analysis'  # Replace with your Google Cloud Storage bucket name
destination_blob_name = 'sf_2018_telecommuting_cleaned_with_telecommuting_rows.csv.gz'  # Replace with the desired destination file name in the bucket

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_file(compressed_bytes, content_type='application/gzip')

# Set the Content-Encoding metadata property
blob.metadata = {'Content-Encoding': 'gzip'}
blob.patch()

CPU times: total: 32min 11s
Wall time: 32min 22s


#### Regression

In [59]:
sf_tel_home_work_small.head()

,IDMerged,tripIndex,actEndTime_24h,actStartTime_24h,duration_travelling_min,actPurpose,cost_BEAM,lever_position_tel,telework,distance_travelling_mile,duration_in_privateCar,duration_in_ridehail,duration_walking,mode_choice_actual_BEAM,replanning_status,mode_choice_actual_4,Potential_INEXUS_in_dollar_2023,Realized_INEXUS_in_dollar_2023,income_quartiles,income_deciles,income_in_thousands,income
0,113,37377.000000,7.840000,7.892500,3.150000,Home_to_work,0.158466,1.000000,False,2.525313,189.000000,0.000000,0.000000,car,0,car,-25.662671,-33.519751,4thD,8thD,160.000000,160000.000000
1,113,37381.000000,22.688889,22.788889,6.000000,work_to_Home,0.370385,1.000000,False,5.902449,360.000000,0.000000,0.000000,car,0,car,-8.014667,-15.737538,4thD,8thD,160.000000,160000.000000
2,113,NaN,0.000000,0.000000,0.000000,Home_to_work,0.000000,1.360000,True,0.000000,0.000000,0.000000,0.000000,telecommuting,0,telecommuting,0.000000,0.000000,4thD,8thD,NaN,NaN
3,113,NaN,0.000000,0.000000,0.000000,work_to_Home,0.000000,1.360000,True,0.000000,0.000000,0.000000,0.000000,telecommuting,0,telecommuting,0.000000,0.000000,4thD,8thD,NaN,NaN
4,113,NaN,0.000000,0.000000,0.000000,Home_to_work,0.000000,1.760000,True,0.000000,0.000000,0.000000,0.000000,telecommuting,0,telecommuting,0.000000,0.000000,4thD,8thD,NaN,NaN


In [60]:
# Get dummy variables for the four columns
dummy_cols = ['lever_position_tel', 'mode_choice_actual_4', 'income_deciles', 'income_quartiles']
sf_tel_home_work_dummies = pd.get_dummies(sf_tel_home_work_small, columns=dummy_cols)

In [64]:
# Group by 'IDMerged' and fill missing values in 'income'
sf_tel_filled = sf_tel_home_work_dummies.groupby('IDMerged').apply(lambda x: x.fillna({'income': x['income'].fillna(x['income'].values[0])}))

# Remove grouping
sf_tel_filled = sf_tel_filled.reset_index(drop=True)

In [66]:
# Group by 'IDMerged' and fill missing values in 'income_in_thousands'
sf_tel_filled2 = sf_tel_filled.groupby('IDMerged').apply(lambda x: x.fillna({'income_in_thousands': x['income_in_thousands'].fillna(x['income_in_thousands'].values[0])}))

# Remove grouping
sf_tel_filled2 = sf_tel_filled2.reset_index(drop=True)

In [90]:
sf_tel_filled2.head()

,IDMerged,tripIndex,actEndTime_24h,actStartTime_24h,duration_travelling_min,actPurpose,cost_BEAM,telework,distance_travelling_mile,duration_in_privateCar,duration_in_ridehail,duration_walking,mode_choice_actual_BEAM,replanning_status,Potential_INEXUS_in_dollar_2023,Realized_INEXUS_in_dollar_2023,income_in_thousands,income,lever_position_tel_1.0,lever_position_tel_1.36,lever_position_tel_1.76,lever_position_tel_2.56,lever_position_tel_4.32,lever_position_tel_6.24,lever_position_tel_9.6,mode_choice_actual_4_car,mode_choice_actual_4_ride_hail,mode_choice_actual_4_telecommuting,mode_choice_actual_4_transit,mode_choice_actual_4_walk/bike,income_deciles_10thD,income_deciles_1stD,income_deciles_2ndD,income_deciles_3rdD,income_deciles_4thD,income_deciles_5thD,income_deciles_6thD,income_deciles_7thD,income_deciles_8thD,income_deciles_9thD,income_quartiles_1stQ,income_quartiles_2ndQ,income_quartiles_3rdQ,income_quartiles_4thD,IDMerged_index
0,113,37377.000000,7.840000,7.892500,3.150000,Home_to_work,0.158466,False,2.525313,189.000000,0.000000,0.000000,car,0,-25.662671,-33.519751,160.000000,160000.000000,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,113,37381.000000,22.688889,22.788889,6.000000,work_to_Home,0.370385,False,5.902449,360.000000,0.000000,0.000000,car,0,-8.014667,-15.737538,160.000000,160000.000000,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,113,NaN,0.000000,0.000000,0.000000,Home_to_work,0.000000,True,0.000000,0.000000,0.000000,0.000000,telecommuting,0,0.000000,0.000000,160.000000,160000.000000,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,113,NaN,0.000000,0.000000,0.000000,work_to_Home,0.000000,True,0.000000,0.000000,0.000000,0.000000,telecommuting,0,0.000000,0.000000,160.000000,160000.000000,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,113,NaN,0.000000,0.000000,0.000000,Home_to_work,0.000000,True,0.000000,0.000000,0.000000,0.000000,telecommuting,0,0.000000,0.000000,160.000000,160000.000000,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [98]:
sf_tel_car = sf_tel_filled2.groupby('IDMerged').filter(lambda x: (x['mode_choice_actual_4_car'] == 1).any())

In [105]:
sf_tel_car.head()

,IDMerged,tripIndex,actEndTime_24h,actStartTime_24h,duration_travelling_min,actPurpose,cost_BEAM,telework,distance_travelling_mile,duration_in_privateCar,duration_in_ridehail,duration_walking,mode_choice_actual_BEAM,replanning_status,Potential_INEXUS_in_dollar_2023,Realized_INEXUS_in_dollar_2023,income_in_thousands,income,lever_position_tel_1.0,lever_position_tel_1.36,lever_position_tel_1.76,lever_position_tel_2.56,lever_position_tel_4.32,lever_position_tel_6.24,lever_position_tel_9.6,mode_choice_actual_4_car,mode_choice_actual_4_ride_hail,mode_choice_actual_4_telecommuting,mode_choice_actual_4_transit,mode_choice_actual_4_walk/bike,income_deciles_10thD,income_deciles_1stD,income_deciles_2ndD,income_deciles_3rdD,income_deciles_4thD,income_deciles_5thD,income_deciles_6thD,income_deciles_7thD,income_deciles_8thD,income_deciles_9thD,income_quartiles_1stQ,income_quartiles_2ndQ,income_quartiles_3rdQ,income_quartiles_4thD,IDMerged_index
0,113,37377.000000,7.840000,7.892500,3.150000,Home_to_work,0.158466,False,2.525313,189.000000,0.000000,0.000000,car,0,-25.662671,-33.519751,160.000000,160000.000000,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,113,37381.000000,22.688889,22.788889,6.000000,work_to_Home,0.370385,False,5.902449,360.000000,0.000000,0.000000,car,0,-8.014667,-15.737538,160.000000,160000.000000,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,113,NaN,0.000000,0.000000,0.000000,Home_to_work,0.000000,True,0.000000,0.000000,0.000000,0.000000,telecommuting,0,0.000000,0.000000,160.000000,160000.000000,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,113,NaN,0.000000,0.000000,0.000000,work_to_Home,0.000000,True,0.000000,0.000000,0.000000,0.000000,telecommuting,0,0.000000,0.000000,160.000000,160000.000000,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,113,NaN,0.000000,0.000000,0.000000,Home_to_work,0.000000,True,0.000000,0.000000,0.000000,0.000000,telecommuting,0,0.000000,0.000000,160.000000,160000.000000,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [104]:
import econtools.metrics as mt

In [125]:
sf_tel_car['income_deciles_1stD_and_lever_position_tel_1'] = sf_tel_car['income_deciles_1stD'] * sf_tel_car['lever_position_tel_1.0']

In [129]:
sf_tel_car['income_deciles_10thD_and_lever_position_tel_1'] = sf_tel_car['income_deciles_10thD'] * sf_tel_car['lever_position_tel_1.0']

In [130]:
sf_tel_car['income_deciles_10thD_and_lever_position_tel_6.24'] = sf_tel_car['income_deciles_10thD'] * sf_tel_car['lever_position_tel_6.24']

In [131]:
sf_tel_car['income_deciles_1stD_and_lever_position_tel_6.24'] = sf_tel_car['income_deciles_1stD'] * sf_tel_car['lever_position_tel_6.24']

In [132]:
y = 'Potential_INEXUS_in_dollar_2023'
X = ['duration_travelling_min', 'lever_position_tel_1.0','lever_position_tel_1.36','lever_position_tel_6.24', 'lever_position_tel_9.6', 'income_deciles_1stD', 'income_deciles_10thD',
    'income_deciles_1stD_and_lever_position_tel_1', 'income_deciles_10thD_and_lever_position_tel_1', 
    'income_deciles_10thD_and_lever_position_tel_6.24', 'income_deciles_1stD_and_lever_position_tel_6.24']
fe_var = 'IDMerged'
cluster_var = 'IDMerged'

results = mt.reg(
    sf_tel_car,             # DataFrame
    y,                      # Dependent var (string)
    X,                      # Independent var(s) (string or list of strings)
    fe_name=fe_var,         # Fixed-effects/absorb var (string)
    cluster=cluster_var     # Cluster var (string)
)
print(results)

C:\Users\nrezaei\anaconda3\envs\BEAM-CORE\lib\site-packages\econtools\util\gentools.py:20: FutureWarning: Explicitly passing `name=None` currently preserves the Series' name or uses a default name of 0. This behaviour is deprecated, and in the future `None` will be used as the name of the resulting DataFrame column.
  return s.to_frame(name)
C:\Users\nrezaei\anaconda3\envs\BEAM-CORE\lib\site-packages\econtools\util\gentools.py:20: FutureWarning: Explicitly passing `name=None` currently preserves the Series' name or uses a default name of 0. This behaviour is deprecated, and in the future `None` will be used as the name of the resulting DataFrame column.
  return s.to_frame(name)


Dependent variable:	Potential_INEXUS_in_dollar_2023
N:			2336321
R-squared:		0.6216
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  IDMerged
  No. of clusters:	  206511
Fixed effects by:	IDMerged
  No. of FE:		  206511
                                                  coeff    se        t   p>t CI_low CI_high
duration_travelling_min                          -0.192 0.001 -139.782 0.000 -0.195  -0.189
lever_position_tel_1.0                           -0.869 0.021  -40.795 0.000 -0.910  -0.827
lever_position_tel_1.36                          -0.491 0.017  -28.926 0.000 -0.524  -0.457
lever_position_tel_6.24                           0.965 0.018   53.182 0.000  0.929   1.000
lever_position_tel_9.6                            1.697 0.019   87.414 0.000  1.659   1.735
income_deciles_1stD                               0.984 0.240    4.105 0.000  0.514   1.453
income_deciles_10thD                              0.841 0.290    2.905 0.004  0.274   1.408
income_deciles_1stD_and_lev

In [134]:
y = 'duration_travelling_min'
X = ['lever_position_tel_1.0','lever_position_tel_1.36','lever_position_tel_6.24', 'lever_position_tel_9.6', 'income_deciles_1stD', 'income_deciles_10thD',
    'income_deciles_1stD_and_lever_position_tel_1', 'income_deciles_10thD_and_lever_position_tel_1', 
    'income_deciles_10thD_and_lever_position_tel_6.24', 'income_deciles_1stD_and_lever_position_tel_6.24']
fe_var = 'IDMerged'
cluster_var = 'IDMerged'

results = mt.reg(
    sf_tel_car,             # DataFrame
    y,                      # Dependent var (string)
    X,                      # Independent var(s) (string or list of strings)
    fe_name=fe_var,         # Fixed-effects/absorb var (string)
    cluster=cluster_var     # Cluster var (string)
)
print(results)

C:\Users\nrezaei\anaconda3\envs\BEAM-CORE\lib\site-packages\econtools\util\gentools.py:20: FutureWarning: Explicitly passing `name=None` currently preserves the Series' name or uses a default name of 0. This behaviour is deprecated, and in the future `None` will be used as the name of the resulting DataFrame column.
  return s.to_frame(name)
C:\Users\nrezaei\anaconda3\envs\BEAM-CORE\lib\site-packages\econtools\util\gentools.py:20: FutureWarning: Explicitly passing `name=None` currently preserves the Series' name or uses a default name of 0. This behaviour is deprecated, and in the future `None` will be used as the name of the resulting DataFrame column.
  return s.to_frame(name)


Dependent variable:	duration_travelling_min
N:			2336321
R-squared:		0.5467
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  IDMerged
  No. of clusters:	  206511
Fixed effects by:	IDMerged
  No. of FE:		  206511
                                                  coeff    se        t   p>t CI_low CI_high
lever_position_tel_1.0                            4.139 0.033  126.590 0.000  4.075   4.203
lever_position_tel_1.36                           2.441 0.026   92.991 0.000  2.389   2.492
lever_position_tel_6.24                          -4.258 0.030 -144.212 0.000 -4.316  -4.200
lever_position_tel_9.6                           -7.206 0.031 -230.141 0.000 -7.268  -7.145
income_deciles_1stD                              -2.043 0.763   -2.678 0.007 -3.539  -0.548
income_deciles_10thD                             -2.211 0.413   -5.353 0.000 -3.020  -1.401
income_deciles_1stD_and_lever_position_tel_1     -2.791 0.117  -23.861 0.000 -3.021  -2.562
income_deciles_10thD_and_lever_posi

In [145]:
y = 'distance_travelling_mile'
X = ['lever_position_tel_1.0','lever_position_tel_1.36','lever_position_tel_6.24', 'lever_position_tel_9.6', 
     'income_deciles_1stD', 'income_deciles_6thD','income_deciles_10thD',
    'income_deciles_1stD_and_lever_position_tel_1', 'income_deciles_10thD_and_lever_position_tel_1', 
    'income_deciles_10thD_and_lever_position_tel_6.24', 'income_deciles_1stD_and_lever_position_tel_6.24']
fe_var = 'IDMerged'
cluster_var = 'IDMerged'

results = mt.reg(
    sf_tel_car,             # DataFrame
    y,                      # Dependent var (string)
    X,                      # Independent var(s) (string or list of strings)
    fe_name=fe_var,         # Fixed-effects/absorb var (string)
    cluster=cluster_var     # Cluster var (string)
)
print(results)

C:\Users\nrezaei\anaconda3\envs\BEAM-CORE\lib\site-packages\econtools\util\gentools.py:20: FutureWarning: Explicitly passing `name=None` currently preserves the Series' name or uses a default name of 0. This behaviour is deprecated, and in the future `None` will be used as the name of the resulting DataFrame column.
  return s.to_frame(name)
C:\Users\nrezaei\anaconda3\envs\BEAM-CORE\lib\site-packages\econtools\util\gentools.py:20: FutureWarning: Explicitly passing `name=None` currently preserves the Series' name or uses a default name of 0. This behaviour is deprecated, and in the future `None` will be used as the name of the resulting DataFrame column.
  return s.to_frame(name)


Dependent variable:	distance_travelling_mile
N:			2336321
R-squared:		0.6944
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  IDMerged
  No. of clusters:	  206511
Fixed effects by:	IDMerged
  No. of FE:		  206511
                                                  coeff    se        t   p>t CI_low CI_high
lever_position_tel_1.0                            2.262 0.017  134.230 0.000  2.229   2.295
lever_position_tel_1.36                           1.304 0.012  108.179 0.000  1.281   1.328
lever_position_tel_6.24                          -2.404 0.016 -146.171 0.000 -2.436  -2.371
lever_position_tel_9.6                           -4.168 0.019 -214.678 0.000 -4.206  -4.130
income_deciles_1stD                              -1.706 0.355   -4.812 0.000 -2.402  -1.011
income_deciles_6thD                              -0.043 0.155   -0.276 0.782 -0.346   0.261
income_deciles_10thD                             -1.379 0.282   -4.896 0.000 -1.932  -0.827
income_deciles_1stD_and_lever_posi

In [ ]:
# selecting only the rows where their baseline purpose is work to home, home to work, or work to work
desired_purposes = ["work_to_Home", "Home_to_work", "work_to_work"]

baseline = sf_stacked_cle[sf_stacked_cle['lever_position_tel'] == 1]
baseline_work_home = baseline[baseline['actPurpose'].isin(desired_purposes)]

persons = sf_stacked_cle.loc[
    sf_stacked_cle[['IDMerged']].isin(baseline_work_home[['IDMerged']]).all(axis=1),
    ['IDMerged']
]

sf_stacked_base_work_home = sf_stacked_cle.merge(persons_trips, on=['IDMerged'], how='inner')

In [ ]:
sf_stacked_work_home = sf_stacked_base_work_home[sf_stacked_base_work_home['actPurpose'].isin(desired_purposes)]

In [ ]:
# Create dummy variables for the 'lever' column
dummy_vars = pd.get_dummies(sf_stacked_work_home['lever_position_tel'], prefix='lever')

In [ ]:
# Concatenate the dummy variables with the original DataFrame
sf_stacked_work_home_reg = pd.concat([sf_stacked_work_home, dummy_vars], axis=1)

In [ ]:
# Perform one-hot encoding on 'income_deciles'
income_quartiles_encoded = pd.get_dummies(sf_stacked_work_home_reg['income_quartiles'], prefix='income_quartile')

# Concatenate the encoded variables with the original DataFrame
sf_stacked_work_home_reg = pd.concat([sf_stacked_work_home_reg, income_quartiles_encoded], axis=1)

In [ ]:
sf_stacked_work_home_reg.columns = sf_stacked_work_home_reg.columns.str.replace('.', '_')

In [ ]:
#formula = 'Potential_INEXUS_in_dollar_2023 ~ lever_1_0 + lever_1_36 + lever_1_76 + lever_2_56 + lever_4_32 + lever_6_24 + lever_9_6 + lever_1_0:income_quartile_4thD + lever_1_36:income_quartile_4thD + lever_1_76:income_quartile_4thD + lever_2_56:income_quartile_4thD + lever_4_32:income_quartile_4thD + lever_6_24:income_quartile_4thD + lever_9_6:income_quartile_4thD + lever_1_0:income_quartile_1stQ + lever_1_36:income_quartile_1stQ + lever_1_76:income_quartile_1stQ + lever_2_56:income_quartile_1stQ + lever_4_32:income_quartile_1stQ + lever_6_24:income_quartile_1stQ + lever_9_6:income_quartile_1stQ + duration_travelling + C(IDMerged)'

In [ ]:
# Define the formula for regression analysis
formula = 'Potential_INEXUS_in_dollar_2023 ~  lever_1_0 + lever_2_56 + lever_9_6 + income_quartile_1stQ + income_quartile_4thD + lever_1_0:income_quartile_1stQ+ lever_1_0:income_quartile_4thD + lever_2_56:income_quartile_1stQ + lever_2_56:income_quartile_4thD + lever_9_6:income_quartile_1stQ + lever_9_6:income_quartile_4thD + duration_travelling + C(IDMerged)'
# Fit the regression model with fixed effects and clustered standard errors
model = sm.formula.ols(formula=formula, data=sf_stacked_work_home_reg).fit(cov_type='cluster', cov_kwds={'groups': sf_stacked_work_home_reg['IDMerged']})

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Define the formula for regression analysis
formula = 'Potential_INEXUS_in_dollar_2023 + duration_travelling + distance_travelling ~ lever_position_tel + income_decile_10thD + income_decile_1stD + income_decile_2ndD+income_decile_3rdD+income_decile_4thD+income_decile_5thD+income_decile_6thD+income_decile_7thD+income_decile_8thD+income_decile_9thD + lever_position_tel:income_decile_9thD + IDMerged'

# Fit the regression model with fixed effects and clustered standard errors
model = sm.formula.ols(formula=formula, data=sf_stacked_work_home_reg).fit(cov_type='cluster', cov_kwds={'groups': sf_stacked_work_home_reg['IDMerged']})

# Print the summary of the regression model
print(model.summary())

In [ ]:
sf_stacked.groupby('lever_position_tel')['tripIndex'].count()

In [ ]:
sf_stacked.groupby(['lever_position_tel', 'mode_choice_actual_BEAM'])['duration_travelling'].mean()

In [ ]:
import matplotlib.cm as cm

# Define the number of 'telework' categories
num_categories = len(sf_stacked['mode_choice_actual_BEAM'].unique())

# Create the Turbo color palette
colors = cm.get_cmap('tab20', num_categories)

# Grouping the data and calculating the average duration of traveling
grouped_data = sf_stacked.groupby(['lever_position_tel', 'telework'])['duration_travelling'].mean().unstack()

# Creating the stacked bar plot
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figsize as per your preference
grouped_data.plot(kind='bar', stacked=False, ax=ax,colormap=colors)

# Adding labels and title
plt.xlabel('Lever Position')
plt.ylabel('Average Duration of Traveling')

legend = plt.legend(title='Telework', bbox_to_anchor=(1.05, 1), loc='upper left')
#plt.tight_layout()  # Adjust the layout to prevent overlapping

plt.show()

In [ ]:
import matplotlib.cm as cm

# Define the number of 'telework' categories
num_categories = len(sf_stacked['mode_choice_actual_BEAM'].unique())

# Create the Turbo color palette
colors = cm.get_cmap('tab20', num_categories)

# Grouping the data and calculating the average duration of traveling
grouped_data = sf_stacked.loc[(sf_stacked['mode_choice_actual_BEAM'] == 'car')].groupby(['lever_position_tel', 'telework'])['distance_travelling'].mean().unstack()

# Creating the stacked bar plot
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figsize as per your preference
grouped_data.plot(kind='bar', stacked=False, ax=ax,colormap=colors)

# Adding labels and title
plt.xlabel('Lever Position')
plt.ylabel('Average Distance of Traveling')

legend = plt.legend(title='Telework', bbox_to_anchor=(1.05, 1), loc='upper left')
#plt.tight_layout()  # Adjust the layout to prevent overlapping

plt.show()

In [ ]:
unique_values = set()  # Set to store unique values

for cell in sf_stacked['vehicleIds']:
    if isinstance(cell, str):  # Check if the cell is a string
        values = [x.strip() for x in cell.split(",")]  # Split cell into individual values
        for value in values:
            if "body" not in value:
                unique_values.add(value)  # Add unique values excluding those starting with "body"
unique_count = len(unique_values)  # Count the unique values

print("Number of unique values: ", unique_count)

In [ ]:
# Convert float values in 'vehicleIds' column to strings
sf_stacked['vehicleIds'] = sf_stacked['vehicleIds'].astype(str)

# Group the data by the "lever_position_tel" column and count unique vehicles within each category
grouped = sf_stacked.groupby('lever_position_tel')['vehicleIds'].apply(lambda x: ','.join(x)).reset_index()
grouped['unique_vehicles'] = grouped['vehicleIds'].apply(lambda x: set([value for value in x.split(',') if "body" not in value]))

# Print the unique vehicles for each category
for index, row in grouped.iterrows():
    #print("Category:", row['lever_position_tel'])
    #print("Unique vehicles:", row['unique_vehicles'])
    print("Number of unique vehicles:", len(row['unique_vehicles']))
    print()

In [ ]:
# Convert float values in 'vehicleIds' column to strings
sf_stacked['vehicleIds'] = sf_stacked['vehicleIds'].astype(str)

# Group the data by the "lever_position_tel" column and count unique vehicles within each category
grouped = sf_stacked.groupby('lever_position_tel')['vehicleIds'].apply(lambda x: len(set([value for sublist in x.str.split(',') for value in sublist if "body" not in value]))).reset_index()

# Print the lever position and the number of unique vehicles
for index, row in grouped.iterrows():
    lever_position = row['lever_position_tel']
    unique_vehicle_count = row['vehicleIds']
    print("Lever Position:", lever_position)
    print("Number of unique vehicles:", unique_vehicle_count)
    print()

In [ ]:
only_car_rh = sf_stacked[(sf_stacked['mode_choice_actual_4']=='car')|(sf_stacked['mode_choice_actual_4']=='ride_hail')]

In [ ]:
# Convert float values in 'vehicleIds' column to strings
only_car_rh['vehicleIds'] = only_car_rh['vehicleIds'].astype(str)

# Group the data by the "lever_position_tel" column and count unique vehicles within each category
grouped = only_car_rh.groupby('lever_position_tel')['vehicleIds'].apply(lambda x: len(set([value for sublist in x.str.split(',') for value in sublist if "body" not in value]))).reset_index()

# Print the lever position and the number of unique vehicles
for index, row in grouped.iterrows():
    lever_position = row['lever_position_tel']
    unique_vehicle_count = row['vehicleIds']
    print("Lever Position:", lever_position)
    print("Number of unique vehicles:", unique_vehicle_count)
    print()

In [ ]:
only_car_rh.groupby('lever_position_tel')['distance_travelling'].mean()

In [ ]:
only_car_rh.groupby('lever_position_tel')['IDMerged'].nunique()

In [ ]:
data_vmt_pmt = {'lever_position_tel': [1, 1.36, 1.76, 2.56, 4.32, 6.24, 9.6],
         'vehicle_num': [426833, 421270, 418733, 415964, 411993, 403767, 394689],
         'mean_distance': [12990.781365,12712.256761, 12584.385886,12481.229332,12269.650379,11883.148158,11473.185084],
         'person_num': [522305,518478,516539,514432,511521,506283,498975]}

In [ ]:
data_vmt_pmt = pd.DataFrame(data_vmt_pmt)

In [ ]:
data_vmt_pmt['vmt'] = data_vmt_pmt['vehicle_num'] * (data_vmt_pmt['mean_distance']*0.000621371)

In [ ]:
data_vmt_pmt['pmt'] = data_vmt_pmt['person_num'] * (data_vmt_pmt['mean_distance']*0.000621371)

In [ ]:
data_vmt_pmt

In [ ]:
sns.set(rc={'figure.figsize':(16,8)})
sns.set_theme(style="whitegrid")
colors = ['#458AFC', '#F88947'] #'#CC3311','#009988', '#F4A582', '#BBCC33'
ax = sns.lineplot(data = data_vmt_pmt, x = 'lever_position_tel', 
                  y = 'vmt', marker='o',
                  markersize=20, ci=None, linewidth = 6, palette=colors)
ax.set_xlabel('Telecommuting Scenario Multiplier', fontsize = 30)
ax.set_ylabel('VMT', fontsize = 30)
ax.yaxis.set_tick_params(labelsize = 20)
ax.xaxis.set_tick_params(labelsize = 20)

#plt.legend(labels = ['Not disabled', 'Disabled'],
         #fontsize='small', fancybox=False, bbox_to_anchor=(1, 0.5), ncol=1, frameon=True, loc = 'best', prop={'size': 40})

In [ ]:
sns.set(rc={'figure.figsize':(16,8)})
sns.set_theme(style="whitegrid")
colors = ['#CC3311'] #'#CC3311','#009988', '#F4A582', '#BBCC33'
ax = sns.lineplot(data = data_vmt_pmt, x = 'lever_position_tel', 
                  y = 'pmt', marker='o',
                  markersize=20, ci=None, linewidth = 6, color=colors[0])
ax.set_xlabel('Telecommuting Scenario Multiplier', fontsize = 30)
ax.set_ylabel('PMT', fontsize = 30)
ax.yaxis.set_tick_params(labelsize = 20)
ax.xaxis.set_tick_params(labelsize = 20)

#plt.legend(labels = ['Not disabled', 'Disabled'],
         #fontsize='small', fancybox=False, bbox_to_anchor=(1, 0.5), ncol=1, frameon=True, loc = 'best', prop={'size': 40})

In [ ]:
import matplotlib.cm as cm

# Define the number of 'telework' categories
num_categories = len(sf_stacked['mode_choice_actual_BEAM'].unique())

# Create the Turbo color palette
colors = cm.get_cmap('tab20', num_categories)

# Grouping the data and calculating the average duration of traveling
grouped_data = sf_stacked.loc[(sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail')|
                             (sf_stacked['mode_choice_actual_BEAM'] == 'ride_hail_pooled')].groupby(['lever_position_tel', 'telework'])['duration_travelling'].mean().unstack()

# Creating the stacked bar plot
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figsize as per your preference
grouped_data.plot(kind='bar', stacked=False, ax=ax,colormap=colors)

# Adding labels and title
plt.xlabel('Lever Position')
plt.ylabel('Average Duration of Traveling')

legend = plt.legend(title='Telework', bbox_to_anchor=(1.05, 1), loc='upper left')
#plt.tight_layout()  # Adjust the layout to prevent overlapping

plt.show()

In [ ]:
import matplotlib.cm as cm

# Define the number of 'telework' categories
num_categories = len(sf_stacked['mode_choice_actual_BEAM'].unique())

# Create the Turbo color palette
colors = cm.get_cmap('tab20', num_categories)

# Grouping the data and calculating the average duration of traveling
grouped_data = sf_stacked.groupby('lever_position_tel')['duration_travelling'].mean()

# Creating the stacked bar plot
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figsize as per your preference
grouped_data.plot(kind='bar', stacked=False, ax=ax,colormap=colors)

# Adding labels and title
plt.xlabel('Lever Position')
plt.ylabel('Average Duration of Traveling')

#legend = plt.legend(title='Telework', bbox_to_anchor=(1.05, 1), loc='upper left')
#plt.tight_layout()  # Adjust the layout to prevent overlapping

plt.show()

In [ ]:
import matplotlib.cm as cm

# Define the number of 'telework' categories
num_categories = len(test4['lever_position_tel'].unique())

# Create the Turbo color palette
colors = cm.get_cmap('tab20', num_categories)

# Grouping the data and calculating the average duration of traveling
grouped_data = test4.groupby(['lever_position_tel', 'telework'])['duration_travelling'].sum().unstack()

# Creating the stacked bar plot
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figsize as per your preference
grouped_data.plot(kind='bar', stacked=True, ax=ax,colormap=colors)

# Adding labels and title
plt.xlabel('Lever Positions')
plt.ylabel('Total Duration of Traveling')

legend = plt.legend(title='Telework', bbox_to_anchor=(1.05, 1), loc='upper left')
#plt.tight_layout()  # Adjust the layout to prevent overlapping

plt.show()

In [ ]:
import matplotlib.cm as cm

# Define the number of 'telework' categories
num_categories = len(test4['lever_position_tel'].unique())

# Create the Turbo color palette
colors = cm.get_cmap('tab20b', num_categories)

# Grouping the data and calculating the average duration of traveling
grouped_data = test4.groupby(['lever_position_tel', 'telework'])['distance_travelling'].sum().unstack()

# Creating the stacked bar plot
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figsize as per your preference
grouped_data.plot(kind='bar', stacked=True, ax=ax,colormap=colors)

# Adding labels and title
plt.xlabel('Lever Positions')
plt.ylabel('Total Distance of Traveling')

legend = plt.legend(title='Telework', bbox_to_anchor=(1.05, 1), loc='upper left')
#plt.tight_layout()  # Adjust the layout to prevent overlapping

plt.show()

In [ ]:
import matplotlib.cm as cm

# Define the number of 'telework' categories
num_categories = len(test4['lever_position_tel'].unique())

# Create the Turbo color palette
colors = cm.get_cmap('tab20c', num_categories)

# Grouping the data and calculating the average duration of traveling
grouped_data = test4.groupby(['lever_position_tel', 'telework'])['fuel_marginal'].sum().unstack()

# Creating the stacked bar plot
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figsize as per your preference
grouped_data.plot(kind='bar', stacked=True, ax=ax,colormap=colors)

# Adding labels and title
plt.xlabel('Lever Positions')
plt.ylabel('Total Marginal Fuel Consumption')

legend = plt.legend(title='Telework', bbox_to_anchor=(1.05, 1), loc='upper left')
#plt.tight_layout()  # Adjust the layout to prevent overlapping

plt.show()

In [ ]:
import matplotlib.cm as cm

# Define the number of 'telework' categories
num_categories = len(test4['mode_choice_actual_BEAM'].unique())

# Create the Turbo color palette
colors = cm.get_cmap('turbo', num_categories)

# Grouping the data and calculating the average duration of traveling
grouped_data = test4.groupby(['lever_position_tel', 'mode_choice_actual_BEAM'])['duration_travelling'].sum().unstack()

# Creating the stacked bar plot
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figsize as per your preference
grouped_data.plot(kind='bar', stacked=True, ax=ax,colormap=colors)

# Adding labels and title
plt.xlabel('Lever Positions')
plt.ylabel('Total Travel Time')

legend = plt.legend(title='Mode Choice', bbox_to_anchor=(1.05, 1), loc='upper left')
#plt.tight_layout()  # Adjust the layout to prevent overlapping

plt.show()

In [ ]:
inc_h_l = test4[(test4['income_deciles'] == '10thD')|(test4['income_deciles'] == '1stD')]

In [ ]:
import matplotlib.cm as cm

# Define the number of 'telework' categories
num_categories = len(inc_h_l['income_deciles'].unique())

# Create the Turbo color palette
colors = cm.get_cmap('turbo', num_categories)

# Grouping the data and calculating the average duration of traveling
grouped_data = inc_h_l.groupby(['lever_position_tel', 'income_deciles'])['duration_travelling'].sum().unstack()

# Creating the stacked bar plot
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figsize as per your preference
grouped_data.plot(kind='bar', stacked=True, ax=ax,colormap=colors)

# Adding labels and title
plt.xlabel('Lever Positions')
plt.ylabel('Total Travel Time')

legend = plt.legend(title='Income', bbox_to_anchor=(1.05, 1), loc='upper left')
#plt.tight_layout()  # Adjust the layout to prevent overlapping

plt.show()

In [ ]:
sf_stacked['mand_non_mand'] = np.where((sf_stacked['mandatoryCat'] == "from_H_to_M")|
                                       (sf_stacked['mandatoryCat'] == "from_M_to_H")|
                                       (sf_stacked['mandatoryCat'] == "from_M_to_M"), 'mand', 'non_mand')

In [ ]:
import matplotlib.cm as cm

# Define the number of 'telework' categories
num_categories = len(sf_stacked['mand_non_mand'].unique())

# Create the Turbo color palette
colors = cm.get_cmap('turbo', num_categories)

# Grouping the data and calculating the average duration of traveling
grouped_data = sf_stacked.loc[sf_stacked['mode_choice_actual_4']=='car'].groupby(['lever_position_tel', 'mand_non_mand'])['duration_travelling'].mean().unstack()

# Creating the stacked bar plot
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figsize as per your preference
grouped_data.plot(kind='bar', stacked=False, ax=ax,colormap=colors)

# Adding labels and title
plt.xlabel('Lever Positions')
plt.ylabel('Sum Travel Time')

legend = plt.legend(title='Trip Type', bbox_to_anchor=(1.05, 1), loc='upper left')
#plt.tight_layout()  # Adjust the layout to prevent overlapping

plt.show()

In [ ]:
import matplotlib.cm as cm

# Define the number of 'telework' categories
num_categories = len(test4['income_deciles'].unique())

# Create the Turbo color palette
colors = cm.get_cmap('turbo', num_categories)

# Grouping the data and calculating the average duration of traveling
grouped_data = test4.groupby(['lever_position_tel', 'income_deciles'])['duration_travelling'].sum().unstack()

# Creating the stacked bar plot
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figsize as per your preference
grouped_data.plot(kind='bar', stacked=True, ax=ax,colormap=colors)

# Adding labels and title
plt.xlabel('Lever Positions')
plt.ylabel('Total Travel Time')

legend = plt.legend(title='Income', bbox_to_anchor=(1.05, 1), loc='upper left')
#plt.tight_layout()  # Adjust the layout to prevent overlapping

plt.show()

In [ ]:
import matplotlib.cm as cm

# Define the number of 'telework' categories
num_categories = len(test4['actPurpose'].unique())

# Create the Turbo color palette
colors = cm.get_cmap('turbo', num_categories)

# Grouping the data and calculating the average duration of traveling
grouped_data = test4.groupby(['lever_position_tel', 'actPurpose'])['duration_travelling'].sum().unstack()

# Creating the stacked bar plot
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figsize as per your preference
grouped_data.plot(kind='bar', stacked=True, ax=ax,colormap=colors)

# Adding labels and title
plt.xlabel('Lever Positions')
plt.ylabel('Total Travel Time')

legend = plt.legend(title='Trip Purpose', bbox_to_anchor=(1.05, 1), loc='upper left')
#plt.tight_layout()  # Adjust the layout to prevent overlapping

plt.show()

In [ ]:
grouped_data = test4.groupby(['telework','lever_position_tel', 'actPurpose']).size().unstack()

In [ ]:
grouped_data_mode = test4.groupby(['telework','lever_position_tel', 'actPurpose','mode_choice_actual_BEAM']).size().unstack()

In [ ]:
test4.groupby('lever_position_tel')['tripIndex'].count()

In [ ]:
test4.groupby(['lever_position_tel','telework'])['tripIndex'].count()

In [ ]:
test4.groupby(['lever_position_tel', 'mode_choice_actual_BEAM'])['tripIndex'].count()

In [ ]:
grouped_data

In [ ]:
grouped_data_mode

In [ ]:
# Grouping the data and calculating the count of occurrences
grouped_data = test4.groupby(['lever_position_tel', 'mode_choice_actual_BEAM', 'telework']).size().unstack()

# Creating the grouped bar plot
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figsize as per your preference

# Determine the number of lever positions and mode choices
num_levers = len(test4['lever_position_tel'].unique())
num_modes = len(test4['mode_choice_actual_BEAM'].unique())

# Calculate the width for each bar
total_width = 0.8
width = total_width / 2
bar_positions = np.arange(num_levers * num_modes)

# Plotting the grouped bars
bars_telework = ax.bar(bar_positions, grouped_data[True], width, label='Telework', color='tab:blue')
bars_not_telework = ax.bar(bar_positions + width, grouped_data[False], width, label='Not Telework', color='tab:orange')

# Adding labels and title
plt.xlabel('Lever Positions and Mode Choices')
plt.ylabel('Count')
plt.title('Counts of Telework Scenario Rates, Mode Choices, and Teleworking Status')

# Adjusting the x-axis ticks and labels
ax.set_xticks(bar_positions + (total_width / 2))
ax.set_xticklabels(grouped_data.index, rotation=90)

# Adding a legend
plt.legend(title='Telework Status')

plt.tight_layout()
plt.show()

#### INEXUS

In [ ]:
plot_inc = sf_stacked[(sf_stacked['duration_walking'] < 3600)&(sf_stacked['distance_walking'] < 5000)]

In [ ]:
from matplotlib.ticker import FuncFormatter
sns.set(rc={'figure.figsize':(20,14)}) # same mode cleaned data
sns.set_context('talk')
sns.set_style("whitegrid", {'gridcolor': 'grainsboro'})
grid = sns.FacetGrid(plot_inc.loc[(plot_inc['income_deciles'] == '1stD')|
                                    (plot_inc['income_deciles'] == '10thD')], 
                     row = 'income_deciles', col_order=['10thD', '1stD'],
                     hue='lever_position_tel', height=5, aspect=1.5, palette = "rocket_r", legend_out=True )
grid.map(sns.kdeplot, 'Potential_INEXUS_in_dollar_2023', bw_adjust =7, linewidth = 1.5)
#grid.set(xlim=(14, sf_stacked_rh_price['Realized_INEXUS_in_dollar'].max()), ylim=(0,0.06))
#grid.add_legend()

# Define function to format y-axis labels as percentages
def to_percent(y, position):
    return "{:.1f}%".format(y * 100)

for ax, title in zip(grid.axes.flat, ['Lowest 10% Income','Highest 10% Income']):
    ax.set_title(title)

grid.set(xlim=(-30,30))

# Format y-axis labels as percentages
formatter = FuncFormatter(to_percent)
ax.yaxis.set_major_formatter(formatter)

plt.legend(title="Ridehail Scenarios", labels = ['Baseline', '136% Telework Trips', '176% Telework Trips','256% Telework Trips','432% Telework Trips',
                     '624% Telework Trips','960% Telework Trips'],
         fontsize='small', fancybox=False, bbox_to_anchor=(1.8, 0.8), ncol=1, frameon=True, loc = 'right', prop={'size': 18})
grid.set_ylabels('Density', size=18)
grid.set_xlabels('Potential INEXUS ($)', size=18)
ax.yaxis.set_tick_params(labelsize = 18)
ax.xaxis.set_tick_params(labelsize = 18)
# Squash down a little and add title to facetgrid    
plt.subplots_adjust(top=1.1)
#grid.fig.suptitle('Realized INEXUS in dollar across income levels in different lever positions')
#plt.savefig('potential_income_2023.svg', dpi = 300, bbox_extra_artists=(ax.legend(['0% Price', '12.5% Price', '25% Price','50% Price','Baseline','175% Price','300% Price', '500% Price','800% Price'], bbox_to_anchor=(1.4, 0.7), ncol=1, frameon=True, loc = 'right', fontsize=18),), bbox_inches='tight')
#plt.savefig('potential_income_2023.png', dpi = 300, bbox_extra_artists=(ax.legend(['0% Price', '12.5% Price', '25% Price','50% Price','Baseline','175% Price','300% Price', '500% Price','800% Price'], bbox_to_anchor=(1.4, 0.7), ncol=1, frameon=True, loc = 'right', fontsize=18),), bbox_inches='tight')

In [ ]:
sf_stacked['socialCarbonCost'] = sf_stacked['emission_marginal']*185 # 51 or 102

In [ ]:
sf_stacked["mode_actual_baseline"] = np.where(sf_stacked['lever_position_tel']==1, sf_stacked["mode_choice_actual_BEAM"], None)

In [ ]:
sf_stacked["mode_actual_baseline"] = sf_stacked.groupby(['IDMerged','tripIndex'])['mode_actual_baseline'].transform(lambda x: x.ffill().bfill())

In [ ]:
same_mode_actual = sf_stacked[sf_stacked['mode_actual_baseline'] == sf_stacked['mode_choice_actual_BEAM']]

In [ ]:
only_non_tel = same_mode_actual[same_mode_actual['telework'] == False]

In [ ]:
only_tel = same_mode_actual[same_mode_actual['telework'] == True]

In [ ]:
%%time
sns.set(rc={'figure.figsize':(20,14)})
sns.set_style("whitegrid", {'gridcolor': 'grainsboro'})
ax = sns.boxplot(x="potential_in_dollar_2023_relative_to_baseline", y="mode_choice_actual_4", order=["ride_hail","car","transit","walk/bike"], 
                 hue="lever_position_tel", hue_order = [9.6, 6.24, 4.32, 2.56, 1.76, 1.36, 1],
                 data=same_mode_actual, palette="rocket", showfliers=False)
ax.set_xlabel('Potential INEXUS in dollar relative to baseline', fontsize = 40)
ax.set_ylabel('Mode', fontsize = 40)
ax.xaxis.set_tick_params(labelsize = 28)
ax.set_yticklabels(['Ridehail', 'Car','Transit','Walk/Bike'], size = 36)
handles, _ = ax.get_legend_handles_labels()
# Specify the order of the levels in the hue variable
hue_order = ['960% Telework Trips',  '624% Telework Trips', '432% Telework Trips', '256% Telework Trips','176% Telework Trips', '136% Telework Trips', 'Baseline',]
# Create a dictionary that maps the levels of the hue variable to the corresponding colors in the palette
palette_dict = dict(zip(hue_order, sns.color_palette("rocket", n_colors=len(hue_order))))
# Create a new list of handles that corresponds to the order of the labels in the legend
handles = [handles[hue_order.index(label)] for label in hue_order]
# Use the new list of handles and the dictionary of colors to create the legend
ax.legend(handles, hue_order, loc="upper left", prop={'size': 36}, markerscale=3, fontsize=24, title_fontsize=30, bbox_to_anchor=(1, 1))
#plt.savefig('box_sameMode_potential.svg', bbox_inches='tight', dpi=300)
#plt.savefig('box_sameMode_potential.png', bbox_inches='tight', dpi = 300)

In [ ]:
%%time
sns.set(rc={'figure.figsize':(20,14)})
sns.set_style("whitegrid", {'gridcolor': 'grainsboro'})
ax = sns.boxplot(x="realized_in_dollar_2023_relative_to_baseline", y="mode_choice_actual_4", order=["ride_hail","car","transit","walk/bike"], 
                 hue="lever_position_tel", hue_order = [9.6, 6.24, 4.32, 2.56, 1.76, 1.36, 1],
                 data=same_mode_actual, palette="YlGn_r", showfliers=False)
ax.set_xlabel('Realized INEXUS in dollar relative to baseline', fontsize = 40)
ax.set_ylabel('Mode', fontsize = 40)
ax.xaxis.set_tick_params(labelsize = 28)
ax.set_yticklabels(['Ridehail', 'Car','Transit','Walk/Bike'], size = 36)
handles, _ = ax.get_legend_handles_labels()
# Specify the order of the levels in the hue variable
hue_order = ['960% Telework Trips',  '624% Telework Trips', '432% Telework Trips', '256% Telework Trips','176% Telework Trips', '136% Telework Trips', 'Baseline',]
# Create a dictionary that maps the levels of the hue variable to the corresponding colors in the palette
palette_dict = dict(zip(hue_order, sns.color_palette("rocket", n_colors=len(hue_order))))
# Create a new list of handles that corresponds to the order of the labels in the legend
handles = [handles[hue_order.index(label)] for label in hue_order]
# Use the new list of handles and the dictionary of colors to create the legend
ax.legend(handles, hue_order, loc="upper left", prop={'size': 36}, markerscale=3, fontsize=24, title_fontsize=30, bbox_to_anchor=(1, 1))
#plt.savefig('box_sameMode_potential.svg', bbox_inches='tight', dpi=300)
#plt.savefig('box_sameMode_potential.png', bbox_inches='tight', dpi = 300)

In [ ]:
%%time
sns.set(rc={'figure.figsize':(20,14)})
sns.set_style("whitegrid", {'gridcolor': 'grainsboro'})
ax = sns.boxplot(x="realized_in_dollar_2023_relative_to_baseline", y="mode_choice_actual_4", order=["ride_hail","car","transit","walk/bike"], 
                 hue="lever_position_tel", hue_order = [9.6, 6.24, 4.32, 2.56, 1.76, 1.36, 1],
                 data=sf_stacked, palette="YlGn_r", showfliers=False)
ax.set_xlabel('Realized INEXUS in dollar relative to baseline', fontsize = 40)
ax.set_ylabel('Mode', fontsize = 40)
ax.xaxis.set_tick_params(labelsize = 28)
ax.set_yticklabels(['Ridehail', 'Car','Transit','Walk/Bike'], size = 36)
handles, _ = ax.get_legend_handles_labels()
# Specify the order of the levels in the hue variable
hue_order = ['960% Telework Trips',  '624% Telework Trips', '432% Telework Trips', '256% Telework Trips','176% Telework Trips', '136% Telework Trips', 'Baseline',]
# Create a dictionary that maps the levels of the hue variable to the corresponding colors in the palette
palette_dict = dict(zip(hue_order, sns.color_palette("rocket", n_colors=len(hue_order))))
# Create a new list of handles that corresponds to the order of the labels in the legend
handles = [handles[hue_order.index(label)] for label in hue_order]
# Use the new list of handles and the dictionary of colors to create the legend
ax.legend(handles, hue_order, loc="upper left", prop={'size': 36}, markerscale=3, fontsize=24, title_fontsize=30, bbox_to_anchor=(1, 1))
#plt.savefig('box_sameMode_potential.svg', bbox_inches='tight', dpi=300)
#plt.savefig('box_sameMode_potential.png', bbox_inches='tight', dpi = 300)

In [ ]:
%%time
sns.set(rc={'figure.figsize':(20,14)})
sns.set_style("whitegrid", {'gridcolor': 'grainsboro'})
ax = sns.boxplot(x="realized_in_dollar_2023_relative_to_baseline", y="mode_choice_actual_4", order=["ride_hail","car","transit","walk/bike"], 
                 hue="telework",
                 data=same_mode_actual, palette="YlGn_r", showfliers=False)
ax.set_xlabel('Realized INEXUS in dollar relative to baseline', fontsize = 40)
ax.set_ylabel('Mode', fontsize = 40)
ax.xaxis.set_tick_params(labelsize = 28)
ax.set_yticklabels(['Ridehail', 'Car','Transit','Walk/Bike'], size = 36)
handles, _ = ax.get_legend_handles_labels()
# Specify the order of the levels in the hue variable
#hue_order = ['960% Telework Trips',  '624% Telework Trips', '432% Telework Trips', '256% Telework Trips','176% Telework Trips', '136% Telework Trips', 'Baseline',]
# Create a dictionary that maps the levels of the hue variable to the corresponding colors in the palette
palette_dict = dict(zip(hue_order, sns.color_palette("rocket", n_colors=len(hue_order))))
# Create a new list of handles that corresponds to the order of the labels in the legend
handles = [handles[hue_order.index(label)] for label in hue_order]
# Use the new list of handles and the dictionary of colors to create the legend
ax.legend(handles, hue_order, loc="upper left", prop={'size': 36}, markerscale=3, fontsize=24, title_fontsize=30, bbox_to_anchor=(1, 1))
#plt.savefig('box_sameMode_potential.svg', bbox_inches='tight', dpi=300)
#plt.savefig('box_sameMode_potential.png', bbox_inches='tight', dpi = 300)

#### Annika's plot

In [ ]:
small_stacked_rh = sf_stacked[['lever_position_tel','income_deciles','IDMerged','tripIndex',
                                       'income', 'realized_in_dollar_2023_relative_to_baseline', 
                                       'Realized_INEXUS_in_dollar_2023', 'potential_in_dollar_2023_relative_to_baseline',
                                       'Potential_INEXUS_in_dollar_2023', 'mode_choice_actual_4', 'duration_door_to_door',
                                       'waitTime_no_replanning','waitTime_replanning', 'mode_choice_planned_BEAM',
                                       'mode_choice_actual_BEAM','mode_actual_baseline']]

In [ ]:
small_stacked_rh['incomeInThousands'] = small_stacked_rh['income']/1000

In [ ]:
person_income = pd.pivot_table(
   small_stacked_rh,
   index=['IDMerged'],
   aggfunc={'incomeInThousands': lambda x: ', '.join(set(x.dropna().astype(str)))}).reset_index() 

In [ ]:
person_income['incomeInThousands'] = person_income['incomeInThousands'].astype(float)

In [ ]:
# Add a column of income ranks
twenty_one_ranks = person_income['incomeInThousands'].quantile([0, 0.048, 0.095, 0.143, 0.191, 0.239, 0.287, 0.335, 0.383, 0.431, 0.479,
                                                                0.527, 0.575, 0.623, 0.671, 0.719, 0.767, 0.815, 0.863, 0.911, 0.959,
                                                                1]).tolist()

In [ ]:
# Add incomeInThousands twenty_one_ranks
conditions  = [(person_income['incomeInThousands'] >= twenty_one_ranks[0]) & (person_income['incomeInThousands'] < twenty_one_ranks[1]), 
               (person_income['incomeInThousands'] >= twenty_one_ranks[1]) & (person_income['incomeInThousands'] < twenty_one_ranks[2]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[2]) & (person_income['incomeInThousands'] < twenty_one_ranks[3]),
               (person_income['incomeInThousands'] >= twenty_one_ranks[3]) & (person_income['incomeInThousands'] < twenty_one_ranks[4]), 
               (person_income['incomeInThousands'] >=  twenty_one_ranks[4]) & (person_income['incomeInThousands'] < twenty_one_ranks[5]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[5]) & (person_income['incomeInThousands'] < twenty_one_ranks[6]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[6]) & (person_income['incomeInThousands'] < twenty_one_ranks[7]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[7]) & (person_income['incomeInThousands'] < twenty_one_ranks[8]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[8]) & (person_income['incomeInThousands'] < twenty_one_ranks[9]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[9]) & (person_income['incomeInThousands'] <= twenty_one_ranks[10]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[10]) & (person_income['incomeInThousands'] <= twenty_one_ranks[11]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[11]) & (person_income['incomeInThousands'] <= twenty_one_ranks[12]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[12]) & (person_income['incomeInThousands'] <= twenty_one_ranks[13]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[13]) & (person_income['incomeInThousands'] <= twenty_one_ranks[14]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[14]) & (person_income['incomeInThousands'] <= twenty_one_ranks[15]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[15]) & (person_income['incomeInThousands'] <= twenty_one_ranks[16]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[16]) & (person_income['incomeInThousands'] <= twenty_one_ranks[17]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[17]) & (person_income['incomeInThousands'] <= twenty_one_ranks[18]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[18]) & (person_income['incomeInThousands'] <= twenty_one_ranks[19]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[19]) & (person_income['incomeInThousands'] <= twenty_one_ranks[20]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[20]) & (person_income['incomeInThousands'] <= twenty_one_ranks[21])]


choices = [0, 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

In [ ]:
person_income['incRank'] = np.select(conditions, choices, default=None)

In [ ]:
%%time
small_stacked_rh = pd.merge(left = small_stacked_rh, right = person_income, how='left', on = ['IDMerged'], suffixes=('', '_drop'))
small_stacked_rh.drop([col for col in small_stacked_rh.columns if 'drop' in col], axis=1, inplace=True)

In [ ]:
lorenz_data_median = pd.pivot_table(
   small_stacked_rh,
   index=['lever_position_tel', 'incRank'],
   aggfunc={'incomeInThousands': np.mean,
           'Potential_INEXUS_in_dollar_2023': np.median,
           'Realized_INEXUS_in_dollar_2023': np.median,
           'realized_in_dollar_2023_relative_to_baseline': np.median,
           'potential_in_dollar_2023_relative_to_baseline': np.median}).reset_index() 

In [ ]:
lorenz_data = pd.pivot_table(
   small_stacked_rh,
   index=['lever_position_tel', 'incRank'],
   aggfunc={'incomeInThousands': np.mean,
           'Potential_INEXUS_in_dollar_2023': np.mean,
           'Realized_INEXUS_in_dollar_2023': np.mean,
           'realized_in_dollar_2023_relative_to_baseline': np.mean,
           'potential_in_dollar_2023_relative_to_baseline': np.mean}).reset_index() 

In [ ]:
lorenz_data.to_csv('C:/Users/nrezaei/Documents/Data/lorenz.csv')

In [ ]:
df_lorenz = pd.read_csv('C:/Users/nrezaei/Documents/Data/lorenz.csv')

In [ ]:
palette = {
    9.6:  '#521e4d',
    6.24: '#8e1d5b',
    4.32: '#ca1a50',
    2.56: '#ee543f',
    1.76: '#f37751',
    1.36: '#f5966d',
    1: '#f7d0b5'}

In [ ]:
lorenz_data['incRank_percent'] = (lorenz_data['incRank'])*5

In [ ]:
import matplotlib.ticker as ticker
sns.set(rc={'figure.figsize':(20,14)})
sns.set_theme(style="whitegrid")
fig, ax = plt.subplots()
ax = sns.lineplot(data = lorenz_data, x = 'incRank_percent', 
                  y = 'potential_in_dollar_2023_relative_to_baseline', hue = 'lever_position_tel', palette = palette, ci=None, linewidth = 6)
ax.set_xlabel('Income Rank (Percentiles of Income)', fontsize = 36) #45
ax.set_ylabel('Potential INEXUS ($) relative to Baseline', fontsize = 36) #45
ax.yaxis.set_tick_params(labelsize = 26) #35
ax.xaxis.set_tick_params(labelsize = 26) #35
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{int(x)}%'))
plt.legend(labels = ['Baseline', '136% Telework Trips', '176% Telework Trips','256% Telework Trips','432% Telework Trips',
                     '624% Telework Trips','960% Telework Trips'],
         fontsize='medium', fancybox=False, bbox_to_anchor=(1, 0.8), ncol=1, frameon=True, loc = 'best', prop={'size': 26}) #35
#plt.savefig('lorenz.svg', format='svg', bbox_extra_artists=(ax.legend(['0% Price', '12.5% Price', '25% Price','50% Price','Baseline','175% Price','300% Price', '500% Price','800% Price'], fancybox=False, bbox_to_anchor=(1, 0.8), fontsize=26),), bbox_inches='tight') #30
#plt.savefig('lorenz.png', dpi=300)